# Topic modeling

| Student Name         | Student-ID |
|----------------------|------------|
| Marco Di Francesco   | 100632815  |
| Loreto García Tejada | 100643862  |
| György Bence Józsa   | 100633270  |
| József-Hunor Jánosi  | 100516724  |
| Sara-Jane Bittner    | 100498554  |

_Learning goal: Processing text data, converting it into a numerical format and performing topic analysis using SVD._

To complete the assignment you are allowed to use the NLTK natural language toolkit.

In [138]:
import re
import nltk
import pandas as pd
from nltk import ngrams
from nltk.corpus import stopwords
from typing import Union, Tuple, Dict, List
from functools import reduce
import numpy as np

In [139]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

a) Load _applications_of_DM.csv_, e.g. with `pandas` in Python. It has titles and text content of Wikipedia articles on data mining. In the following tasks, you are to work with the "tex"-attribute.

In [140]:
df = pd.read_csv('applications_of_DM.csv')
df

,title,text
0,Anomaly Detection at Multiple Scales,"Anomaly Detection at Multiple Scales, or ADAMS..."
1,Behavioral analytics,Behaviorism is a systematic approach to unders...
2,Business analytics,Business analysis is a professional discipline...
3,CORE (research service),CORE (Connecting Repositories) is a service pr...
4,Daisy Intelligence,Daisy Intelligence is a Canadian Artificial In...
5,Data Applied,Data Applied is a software vendor headquartere...
6,Data mining in agriculture,Data mining in agriculture is a recent researc...
7,Data thinking,"Data thinking is a buzzword for the generic ""m..."
8,Document processing,Document processing is a field of research and...
9,Equifax Workforce Solutions,"Equifax Workforce Solutions, formerly known as..."


b) Process and tokenize the text data: lowercase all words, remove digits, punctuation, any special characters and NLTK’s common English stopwords. List $10$ most frequent $n$-grams, $n = 1, 2, 3,$ and their raw counts. Then, convert $1700$ most frequent $n$-grams, across all $n$-values, into numerical matrices, i.e., features $X_\mathrm{raw},X_\mathrm{tf-idf} \in \mathbb{R}^{docs \times terms}$. Features $X_\mathrm{raw}$ are raw $n$-gram counts, $X_\mathrm{tf-idf}$ are tf-idf values (see the specific format below).

In [141]:
def process_text(text: str) -> list[str]:
    """Removes special characters and numbers from a given text and returns a list of words of that text in lower case, removing common english stopwords.

    :param text: the raw text to be processed
    :return: a list of words in the original text with no special characters, numbers and common english stopwords in lower case.
    """
    # remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    words = text.split()
    # filter out common english stopwords and return list
    return [word for word in words if word not in stopwords.words('english')]

In [142]:
df['filtered_text'] = df['text'].map(process_text)
df

,title,text,filtered_text
0,Anomaly Detection at Multiple Scales,"Anomaly Detection at Multiple Scales, or ADAMS...","[anomaly, detection, multiple, scales, adams, ..."
1,Behavioral analytics,Behaviorism is a systematic approach to unders...,"[behaviorism, systematic, approach, understand..."
2,Business analytics,Business analysis is a professional discipline...,"[business, analysis, professional, discipline,..."
3,CORE (research service),CORE (Connecting Repositories) is a service pr...,"[core, connecting, repositories, service, prov..."
4,Daisy Intelligence,Daisy Intelligence is a Canadian Artificial In...,"[daisy, intelligence, canadian, artificial, in..."
5,Data Applied,Data Applied is a software vendor headquartere...,"[data, applied, software, vendor, headquartere..."
6,Data mining in agriculture,Data mining in agriculture is a recent researc...,"[data, mining, agriculture, recent, research, ..."
7,Data thinking,"Data thinking is a buzzword for the generic ""m...","[data, thinking, buzzword, generic, mental, pa..."
8,Document processing,Document processing is a field of research and...,"[document, processing, field, research, set, p..."
9,Equifax Workforce Solutions,"Equifax Workforce Solutions, formerly known as...","[equifax, workforce, solutions, formerly, know..."


In [143]:
def make_ngram(corpus: list[str], n: int) -> Dict[Tuple[str, ...], int]:
    """Creates a frequency-list of specified n-grams in the given corpus

    :param corpus: Ordered list of words in the corpus
    :param n: number of elements in n-gram creation
    :return: dictionary with keys being unique n-grams and values being the frequencies of them
    """
    grams = list(ngrams(corpus, n))
    return {i:grams.count(i) for i in set(grams)}

In [144]:
df['unigrams'] = df['filtered_text'].map(lambda x: make_ngram(x, n=1))
df['bigrams'] = df['filtered_text'].map(lambda x: make_ngram(x, n=2))
df['trigrams'] = df['filtered_text'].map(lambda x: make_ngram(x, n=3))
df

,title,text,filtered_text,unigrams,bigrams,trigrams
0,Anomaly Detection at Multiple Scales,"Anomaly Detection at Multiple Scales, or ADAMS...","[anomaly, detection, multiple, scales, adams, ...","{('innovation',): 1, ('large',): 1, ('employee...","{('intended', 'recipients'): 1, ('bader', 'cyb...","{('commercial', 'applications', 'may'): 1, ('b..."
1,Behavioral analytics,Behaviorism is a systematic approach to unders...,"[behaviorism, systematic, approach, understand...","{('seminal',): 1, ('large',): 2, ('distinct',)...","{('following', 'controlling'): 1, ('school', '...","{('based', 'instructions', 'later'): 1, ('skin..."
2,Business analytics,Business analysis is a professional discipline...,"[business, analysis, professional, discipline,...","{('large',): 2, ('appear',): 2, ('blue',): 1, ...","{('development', 'solutions'): 1, ('end', 'bus...","{('state', 'requirements', 'solutionthe'): 1, ..."
3,CORE (research service),CORE (Connecting Repositories) is a service pr...,"[core, connecting, repositories, service, prov...","{('repositories',): 6, ('download',): 1, ('lar...","{('data', 'dumps'): 1, ('three', 'access'): 1,...","{('core', 'search', 'engine'): 1, ('first', 'd..."
4,Daisy Intelligence,Daisy Intelligence is a Canadian Artificial In...,"[daisy, intelligence, canadian, artificial, in...","{('grocers',): 1, ('fraudulent',): 1, ('techno...","{('artificial', 'intelligence'): 1, ('companie...","{('ontario', 'downtown', 'toronto'): 1, ('inte..."
5,Data Applied,Data Applied is a software vendor headquartere...,"[data, applied, software, vendor, headquartere...","{('time',): 1, ('founded',): 1, ('intelligence...","{('including', 'visual'): 1, ('association', '...","{('headquartered', 'washington', 'founded'): 1..."
6,Data mining in agriculture,Data mining in agriculture is a recent researc...,"[data, mining, agriculture, recent, research, ...","{('extensive',): 1, ('pictures',): 1, ('belts'...","{('along', 'meteorological'): 1, ('cure', 'ani...","{('decision', 'making', 'thus'): 1, ('reasons'..."
7,Data thinking,"Data thinking is a buzzword for the generic ""m...","[data, thinking, buzzword, generic, mental, pa...","{('landscape',): 1, ('earlier',): 1, ('innovat...","{('combined', 'phase'): 1, ('high', 'business'...","{('stage', 'user', 'customer'): 1, ('develop',..."
8,Document processing,Document processing is a field of research and...,"[document, processing, field, research, set, p...","{('images',): 4, ('ipa',): 1, ('newspaper',): ...","{('automation', 'document'): 1, ('resort', 'de...","{('whether', 'form', 'classical'): 1, ('may', ..."
9,Equifax Workforce Solutions,"Equifax Workforce Solutions, formerly known as...","[equifax, workforce, solutions, formerly, know...","{('accused',): 2, ('district',): 1, ('large',)...","{('offered', 'shares'): 1, ('headquartered', '...","{('price', 'cohen', 'sold'): 1, ('founded', 'n..."


In [145]:
def merge_frequency_dictionaries(first: Dict[Tuple[str, ...], int], second: Dict[Tuple[str, ...], int]) -> Dict[Tuple[str, ...], int]:
    """Merges two dictionaries of frequencies

    :param first: first dictionary containing frequencies
    :param second: second dictionary containing frequencies
    :return: a merged dictionary
    """
    res = {}
    for key, value in first.items():
        res[key] = value
    for key, value in second.items():
        if key in res:
            res[key] += value
        else:
            res[key] = value
    return res

In [146]:
def combine(series):
    """ Combines a list of dictionaries into a single dictionary

    :param series: a list of dictionaries
    :return: a single dictionary
    """
    return reduce(lambda x,y: merge_frequency_dictionaries(x,y), series)

In [147]:
def get_most_frequent(grams: pd.Series, n: int) -> list[Tuple[int, Tuple[str, ...]]]:
    """ Returns the n most frequent n-grams

    :param grams: a list of dictionaries
    :param n: the number of n-grams to return
    :return: a list of tuples containing the frequency and the n-gram
    """
    combined_grams = grams.aggregate(combine)
    return sorted([(v, k) for k, v in combined_grams.items()], reverse=True)[:n]

In [148]:
get_most_frequent(df['unigrams'], 10)

[(160, ('behavior',)),
 (156, ('analysis',)),
 (150, ('data',)),
 (148, ('business',)),
 (103, ('also',)),
 (102, ('text',)),
 (86, ('anpr',)),
 (85, ('use',)),
 (84, ('used',)),
 (84, ('plate',))]

In [149]:
get_most_frequent(df['bigrams'], 10)


[(43, ('text', 'mining')),
 (43, ('license', 'plate')),
 (33, ('business', 'analysis')),
 (22, ('data', 'mining')),
 (20, ('open', 'access')),
 (19, ('document', 'processing')),
 (17, ('business', 'analysts')),
 (17, ('anpr', 'systems')),
 (16, ('radical', 'behaviorism')),
 (16, ('behavior', 'analysis'))]

In [150]:
get_most_frequent(df['trigrams'], 10)


[(8, ('number', 'plate', 'recognition')),
 (7, ('b', 'f', 'skinner')),
 (7, ('automatic', 'number', 'plate')),
 (6, ('optical', 'character', 'recognition')),
 (6, ('license', 'plate', 'capture')),
 (6, ('average', 'speed', 'cameras')),
 (5, ('text', 'data', 'mining')),
 (5, ('open', 'access', 'content')),
 (5, ('natural', 'language', 'processing')),
 (4, ('text', 'mining', 'software'))]

In [152]:
for ngram in ["unigrams", "bigrams", "trigrams"]:
    mfw = get_most_frequent(df[ngram], 1700)
    for doc_idx, doc in df.iterrows():
        arr = np.zeros([17, 1700])
        mfw_doc = doc[ngram]
        words_ls = []
        for w_idx, (_, words) in enumerate(mfw):
            words_ls.append(words)
            count = mfw_doc[words] if words in mfw_doc else 0
            arr[doc_idx][w_idx] = count
            # print(words)
        print(pd.DataFrame(arr, columns=words_ls))

    (behavior,)  (analysis,)  (data,)  (business,)  (also,)  (text,)  (anpr,)  \
0           0.0          1.0      1.0          0.0      0.0      0.0      0.0   
1           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
2           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
3           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
4           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
5           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
6           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
7           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
8           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
9           0.0          0.0      0.0          0.0      0.0      0.0      0.0   
10          0.0          0.0      0.0          0.0      0.0      0.0      0.0   
11          0.0          0.0

c) Using SVD, decompose and truncate your numerical features $X = U \Sigma V^\top$ into ($docs \times topics$) and ($topics \times terms$) matrices (left/right singular vectors, respectively) using $6$ topics. List $5$ most significant $n$-grams for each topic, measured by values of the ($topics \times terms$) matrix. Do this for both features $X_\mathrm{raw}$ and $X_\mathrm{tf-idf}$.

d) Compare and comment the results with respect to the selection of features and the $n$ value in $n$-grams.

**The format of tf-idf you need to use:**

$$ f(tf, idf) = (1 + \ln (tf)) \cdot idf $$

where:

$$ tf = \frac{number\ of\ times\ term\ w\ appears\ in\ a\ document}{total\ number\ of\ terms\ in\ that\ document} $$

and

$$ idf = \ln(\frac{total\ number\ of\ documents}{number\ of\ documents\ with\ term\ w\ in\ them + 1} + 1) $$